In [1]:
import pandas as pd
import numpy as np
import data

In [2]:
bfield_data = pd.read_csv(data.DIR_PATH / "bender" / "FA_Bield_Bender_Iso_100large.csv")
bfield_data

,x,y,z,Bx,By,Bz
0,-0.040000,-0.040000,-0.219236,0.000033,1.215512e-05,-7.284397e-06
1,-0.040000,-0.040000,-0.217767,0.000042,2.031730e-05,-3.869628e-05
2,-0.040000,-0.040000,-0.216298,0.000049,2.702022e-05,-7.753445e-05
3,-0.040000,-0.040000,-0.214829,0.000054,3.135252e-05,-1.246384e-04
4,-0.040000,-0.040000,-0.213360,0.000056,3.218574e-05,-1.807521e-04
...,...,...,...,...,...,...
604995,0.039333,0.039333,0.067243,-0.000002,9.782542e-07,-1.971002e-07
604996,0.039333,0.039333,0.068712,-0.000002,9.580174e-07,-2.094007e-07
604997,0.039333,0.039333,0.070181,-0.000002,9.382850e-07,-2.209203e-07
604998,0.039333,0.039333,0.071651,-0.000002,9.190423e-07,-2.317022e-07


In [17]:
# Step 1: Extract unique coordinates and sort
x_vals = np.sort(bfield_data['x'].unique())
y_vals = np.sort(bfield_data['y'].unique())
z_vals = np.sort(bfield_data['z'].unique())

dx = x_vals[1] - x_vals[0]
dy = y_vals[1] - y_vals[0]
dz = z_vals[1] - z_vals[0]

# Step 2: Reshape the data into 3D grids
Nx, Ny, Nz = len(x_vals), len(y_vals), len(z_vals)
print(f"Grid shape: {Nx} x {Ny} x {Nz}")

# Must sort the dataframe by (x, y, z) to match the reshape order
bfield_data_sorted = bfield_data.sort_values(['x', 'y', 'z'])

Bx = bfield_data_sorted['Bx'].to_numpy().reshape(Nx, Ny, Nz)
By = bfield_data_sorted['By'].to_numpy().reshape(Nx, Ny, Nz)
Bz = bfield_data_sorted['Bz'].to_numpy().reshape(Nx, Ny, Nz)

# Step 3: Apply gradient
dBx_dx, dBx_dy, dBx_dz = np.gradient(Bx, dx, dy, dz, edge_order=2)
dBy_dx, dBy_dy, dBy_dz = np.gradient(By, dx, dy, dz, edge_order=2)
dBz_dx, dBz_dy, dBz_dz = np.gradient(Bz, dx, dy, dz, edge_order=2)

# Step 4: Construct full Jacobian ∇B as shape (Nx, Ny, Nz, 3, 3)


Grid shape: 55 x 55 x 200


In [16]:
dBx_dx.shape

(55, 55, 200)

In [ ]:
gradB = np.stack([
    np.stack([dBx_dx, dBx_dy, dBx_dz], axis=-1),
    np.stack([dBy_dx, dBy_dy, dBy_dz], axis=-1),
    np.stack([dBz_dx, dBz_dy, dBz_dz], axis=-1)
], axis=-2)

gradB.shape

In [8]:
# The data contains x,y,z, Bx, By, Bz, and I want to take the gradient of the vector field B:

gradient = np.gradient(
    bfield_data[["Bx", "By", "Bz"]].to_numpy(),

)
gradient[0]



array([[ 8.89755263e-06,  8.16218225e-06, -3.14118805e-05],
       [ 8.18193854e-06,  7.43255100e-06, -3.51250273e-05],
       [ 6.29522751e-06,  5.51761019e-06, -4.29710734e-05],
       ...,
       [ 4.16318447e-08, -1.94875672e-08, -1.11507616e-08],
       [ 4.05281256e-08, -1.90049821e-08, -1.04331340e-08],
       [ 3.99854990e-08, -1.87672407e-08, -1.00844288e-08]],
      shape=(605000, 3))

In [9]:
gradient[1]


array([[-2.04673281e-05, -1.99534210e-05, -1.94395138e-05],
       [-2.12026985e-05, -4.01081375e-05, -5.90135766e-05],
       [-2.19661032e-05, -6.32603868e-05, -1.04554670e-04],
       ...,
       [ 3.03290665e-06,  9.36850665e-07, -1.15920532e-06],
       [ 2.97259318e-06,  9.10924369e-07, -1.15074444e-06],
       [ 2.91384044e-06,  8.85889405e-07, -1.14206163e-06]],
      shape=(605000, 3))